---

## Reinforcement Learning Final Project

**File:** `main.ipynb`  
**Author:** hamid manouchehri & Christ Joe Maria Anantharaj
 
**Email:** hmanouch@buffalo.edu & christjo@buffalo.edu
**Date:** Apr 10, 2025

---

### Description:
Robotic manipulator on pybullet simulator for gripping an inhomogeneous object 
from its COM to avoid tilting while lifting the object. 

---

### License:
This script is licensed under the **MIT License**.  
You may obtain a copy of the License at:  
🔗 [https://opensource.org/licenses/MIT](https://opensource.org/licenses/MIT)

**SPDX-License-Identifier:** MIT

---

### Disclaimer:
> This software is provided "as is", without warranty of any kind, express or implied, including but not limited to the warranties of merchantability, fitness for a particular purpose, and noninfringement.  
> In no event shall the authors be liable for any claim, damages, or other liability, whether in an action of contract, tort, or otherwise, arising from, out of, or in connection with the software or the use or other dealings in the software.

---

In [1]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pybullet as p
import pybullet_data
import time
import matplotlib.pyplot as plt
from attrdict import AttrDict

pybullet build time: Jan 29 2025 23:19:04


### Intitial loading of the pybullet environment

In [2]:
# Connect to PyBullet and set up the environment
p.connect(p.GUI)
p.resetSimulation()
p.setAdditionalSearchPath(pybullet_data.getDataPath())
# p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)  # Hides the PyBullet GUI

# Load the KUKA robot and environment objects
planeId = p.loadURDF("plane.urdf")
cuboid_green_id = p.loadURDF("./object/block.urdf", [0.54, 0, 0.02], [0, 0, 0, 1])
kuka_id = p.loadURDF("kuka_iiwa/kuka_with_prismatic_gripper.urdf")

p.setGravity(0, 0, -10)
p.resetDebugVisualizerCamera( # Set the camera view
    cameraDistance=2.0,    # Zoom level (2 meters away)
    cameraYaw=75,          # 45 degrees rotation around the vertical axis
    cameraPitch=-40,       # Tilt 30 degrees downward
    cameraTargetPosition=[0, 0, 0]  # Focus on the origin (0, 0, 0)
)

sim_time_step = 1 / 240  # TODO
eff_index = 7

numJoints = p.getNumJoints(kuka_id)  # kuka + gripper

# Initialize the joints dictionary
joints = AttrDict()

# Populate the joints dictionary with information about each joint
for joint_index in range(p.getNumJoints(kuka_id)):
    joint_info = p.getJointInfo(kuka_id, joint_index)
    joint_name = joint_info[1].decode("utf-8")
    joints[joint_name] = AttrDict({
        "id": joint_info[0],
        "lowerLimit": joint_info[8],
        "upperLimit": joint_info[9],
        "maxForce": joint_info[10],
        "maxVelocity": joint_info[11],
    })

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) Graphics (ADL GT2)
GL_VERSION=4.6 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 21.2.6
Vendor = Intel
Renderer = Mesa Intel(R) Graphics (ADL GT2)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu


### Environment

In [ ]:
class BulletKukaEnvironment(gym.Env):
    # def __init__(self, gui=True):
    #     # Connect to PyBullet
    #     self.physicsClient = p.connect(p.GUI if gui else p.DIRECT)
    #     p.setAdditionalSearchPath(pybullet_data.getDataPath())
    #     p.setGravity(0, 0, -10)
        
    #     # URDF paths
    #     self.plane_id = None
    #     self.robot_path = "kuka_iiwa/kuka_with_prismatic_gripper.urdf"
    #     self.object_path = "./object/block.urdf" 
        
    #     self.robot_id = None
    #     self.object_id = None
    #     self.end_effector_index = 7 
        
    #     # Simulation parameters
    #     self.sim_time_step = 1/240
    #     p.setTimeStep(self.sim_time_step)
        
    #     # Observation space
    #     self.observation_space = spaces.Box(
    #         low=np.array([0.3, -1.0]),
    #         high=np.array([0.7, 1.0]),
    #         dtype=np.float32
    #     )
        
    #     # Action space:
    #     self.action_space = spaces.Box(
    #         low=np.array([-0.1]),  # move left
    #         high=np.array([0.1]),  # move right
    #         dtype=np.float32
    #     )
        
    #     self.steps = 0
    #     self.max_steps = 100
    #     self.current_grip_x = 0.5 
    #     self.true_com_x = 0.5 
    #     self.joints = None  
        
    #     # Initialize the environment
    #     self.reset()

    # def reset(self):
    #     p.resetSimulation()
    #     p.setGravity(0, 0, -10)
    #     p.setTimeStep(self.sim_time_step)

    #     # Load the environment
    #     self.plane_id = p.loadURDF("plane.urdf")
    #     self.robot_id = p.loadURDF(self.robot_path, [0, 0, 0], useFixedBase=1)
        
    #     # Setup camera view
    #     p.resetDebugVisualizerCamera(
    #         cameraDistance=1.5,
    #         cameraYaw=75,
    #         cameraPitch=-40,
    #         cameraTargetPosition=[0, 0, 0]
    #     )
        
    #     # Initialize joints dictionary
    #     self.joints = AttrDict()
    #     for joint_index in range(p.getNumJoints(self.robot_id)):
    #         joint_info = p.getJointInfo(self.robot_id, joint_index)
    #         joint_name = joint_info[1].decode("utf-8")
    #         self.joints[joint_name] = AttrDict({
    #             "id": joint_info[0],
    #             "lowerLimit": joint_info[8],
    #             "upperLimit": joint_info[9],
    #             "maxForce": joint_info[10],
    #             "maxVelocity": joint_info[11],
    #         })
        
    #     # Randomize the true center of mass of the object
    #     self.true_com_x = np.random.uniform(0.4, 0.6)
        
    #     # Place the object on plane
    #     self.object_id = p.loadURDF(
    #         self.object_path, 
    #         [self.true_com_x, 0, 0.02],  
    #         [0, 0, 0, 1]  
    #     )
        
    #     # Randomize initial grip position
    #     self.current_grip_x = np.random.uniform(0.3, 0.7)
        
    #     # Move robot to initial position 
    #     init_joint_angles = np.array([0.]*7)
    #     des_joint_angles = np.array([0., 0.114, 0., -1.895, 0., 1.13, 0.])
    #     self.set_kuka_joint_angles(init_joint_angles, des_joint_angles, duration=1)
        
    #     # Open gripper
    #     self.execute_gripper(init_pos=0., fin_pos=0.01, duration=0.5)
        
    #     for _ in range(50):
    #         p.stepSimulation()

    #     self.steps = 0
    #     return self.get_observation()

    def calculate_ik(self, position, orientation):
        quaternion = p.getQuaternionFromEuler(orientation)
        # print(quaternion)
        # quaternion = (0,1,0,1)
        lower_limits = [-np.pi]*7
        upper_limits = [np.pi]*7
        joint_ranges = [2*np.pi]*7
        rest_poses = [(-0.0, -0.0, 0.0, -0.0, -0.0, 0.0, 0.0)] # rest pose of our ur5 robot

        joint_angles = p.calculateInverseKinematics(
            kuka_id, eff_index, position, quaternion, 
            jointDamping=[0.01]*10 , upperLimits=upper_limits, 
            lowerLimits=lower_limits, jointRanges=joint_ranges, 
            restPoses=rest_poses
        )
        return joint_angles


    def set_kuka_joint_angles(self, init_joint_angles, des_joint_angles, duration):
        control_joints = ["lbr_iiwa_joint_1", "lbr_iiwa_joint_2", "lbr_iiwa_joint_3",
                        "lbr_iiwa_joint_4", "lbr_iiwa_joint_5", "lbr_iiwa_joint_6",
                        "lbr_iiwa_joint_7"]
        poses = []
        indexes = []
        forces = []

        for i, name in enumerate(control_joints):
            joint = joints[name]
            poses.append(des_joint_angles[i])
            indexes.append(joint.id)
            forces.append(joint.maxForce)

        trajectory = self.interpolate_trajectory(init_joint_angles, des_joint_angles, duration)

        for q_t in trajectory:
            p.setJointMotorControlArray(
                kuka_id, indexes,
                controlMode=p.POSITION_CONTROL,
                targetPositions=q_t,
                forces=forces
            )
            # print("tilt: ", get_object_state(cuboid_green_id)[1])
            p.stepSimulation()
            time.sleep(sim_time_step)  # Match real-time simulatio


    def position_path(self, t, t_max, start_pos, end_pos):

        return start_pos + (end_pos - start_pos) * (t / t_max)


    def orientation_path(self, t, t_max, start_orient, end_orient):
        """ orientation is in Euler. """

        return start_orient + (end_orient - start_orient) * (t / t_max)


    def get_current_eff_pose(self):
        linkstate = p.getLinkState(kuka_id, eff_index, computeForwardKinematics=True)
        position, orientation = linkstate[0], linkstate[1]
        return (position, p.getEulerFromQuaternion(orientation))


    def get_current_joint_angles(self, kuka_or_gripper=None):
        "revolute joints: kuka + gripper"
        j = p.getJointStates(kuka_id, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
        joints = [i[0] for i in j]

        if kuka_or_gripper == 'kuka':
            return joints[:7]  # [0, 1, 2, 3, 4, 5, 6]
        
        elif kuka_or_gripper == 'gripper':
            return joints[8:]  # [8, 9]

        else:
            return joints


    def execute_task_space_trajectory(self, start_pos, final_pos, duration=1):

        all_joint_angles = self.calculate_ik(final_pos[0], final_pos[1])
        des_kuka_joint_angles = all_joint_angles[:7]  # Use the first 6 joints for UR5
        self.set_kuka_joint_angles(self.get_current_joint_angles("kuka"), des_kuka_joint_angles, duration)



    def execute_gripper(self, init_pos, fin_pos, duration=1):
        """
        Smoothly open or close the gripper by interpolating the gripper opening angle.

        Args:
            time_step (float): Time increment for each simulation step.
            duration (float): Total duration for the gripper motion (in seconds).
            init_pos (float): Initial gripper opening distance.
            fin_pos (float): Final gripper opening distance.
        """
        control_joints = ["left_finger_sliding_joint", "right_finger_sliding_joint"]

        poses = []
        indexes = []
        forces = []

        init_pos = np.array([-init_pos, init_pos])
        fin_pos = np.array([-fin_pos, fin_pos])

        for i, name in enumerate(control_joints):
            joint = joints[name]
            poses.append(fin_pos[i])
            indexes.append(joint.id)
            forces.append(joint.maxForce)

        trajectory = self.interpolate_trajectory(init_pos, fin_pos, duration)

        for q_t in trajectory:
            p.setJointMotorControlArray(
                kuka_id, indexes,
                controlMode=p.POSITION_CONTROL,
                targetPositions=q_t,
                forces=forces
            )
            # print("tilt: ", get_object_state(cuboid_green_id)[1])
            p.stepSimulation()
            time.sleep(sim_time_step)  # Match real-time simulatio


    def get_object_state(self, object_id):

        position, orientation = p.getBasePositionAndOrientation(object_id)
        orientation_euler = p.getEulerFromQuaternion(orientation)
        linear_velocity, angular_velocity = p.getBaseVelocity(object_id)

        return orientation_euler


    def interpolate_trajectory(self, q_start, q_end, duration):
        num_steps = int(duration / sim_time_step)
        trajectory = []
        for t in range(num_steps):
            alpha = t / (num_steps - 1)  # Normalized time [0, 1]
            q_t = [q_start[i] + alpha * (q_end[i] - q_start[i]) for i in range(len(q_start))]
            trajectory.append(q_t)

        return trajectory 
    
    def step(self, action):

        modified_pos_eef = [list(row) for row in self.get_current_eff_pose()]
        x_step = .02
        z_step = .13

        if action == "xPlus":
            modified_pos_eef[0][0] = self.get_current_eff_pose()[0][0] + x_step
            self.execute_task_space_trajectory(self.get_current_eff_pose(), modified_pos_eef, duration=1)
            self.execute_gripper(init_pos=0.0, fin_pos=0.01, duration=1)  # open gripper
            modified_pos_eef[0][2] = self.get_current_eff_pose()[0][2] - z_step
            self.execute_task_space_trajectory(self.get_current_eff_pose(), modified_pos_eef, duration=1)
            self.execute_gripper(init_pos=0.01, fin_pos=0.0008, duration=0.5)  # close gripper
            modified_pos_eef[0][2] = self.get_current_eff_pose()[0][2] + z_step         
            self.execute_task_space_trajectory(self.get_current_eff_pose(), modified_pos_eef, duration=1)
            tilt_angle = self.get_object_state(cuboid_green_id)[1]
            modified_pos_eef[0][2] = self.get_current_eff_pose()[0][2] - z_step
            self.execute_task_space_trajectory(self.get_current_eff_pose(), modified_pos_eef, duration=1)
            self.execute_gripper(init_pos=0.0, fin_pos=0.01, duration=0.5)  # open gripper
            modified_pos_eef[0][2] = self.get_current_eff_pose()[0][2] + z_step
            self.execute_task_space_trajectory(self.get_current_eff_pose(), modified_pos_eef, duration=1)

        elif action == "xMinus":
            modified_pos_eef[0][0] = self.get_current_eff_pose()[0][0] - x_step
            self.execute_task_space_trajectory(self.get_current_eff_pose(), modified_pos_eef, duration=1)
            self.execute_gripper(init_pos=0.0, fin_pos=0.01, duration=1)  # open gripper
            modified_pos_eef[0][2] = self.get_current_eff_pose()[0][2] - z_step
            self.execute_task_space_trajectory(self.get_current_eff_pose(), modified_pos_eef, duration=1)
            self.execute_gripper(init_pos=0.01, fin_pos=0.00085, duration=0.5)  # close gripper
            modified_pos_eef[0][2] = self.get_current_eff_pose()[0][2] + z_step          
            self.execute_task_space_trajectory(self.get_current_eff_pose(), modified_pos_eef, duration=1)
            tilt_angle = self.get_object_state(cuboid_green_id)[1]
            modified_pos_eef[0][2] = self.get_current_eff_pose()[0][2] - z_step
            self.execute_task_space_trajectory(self.get_current_eff_pose(), modified_pos_eef, duration=1)
            self.execute_gripper(init_pos=0.0, fin_pos=0.01, duration=0.5)  # open gripper
            modified_pos_eef[0][2] = self.get_current_eff_pose()[0][2] + z_step          
            self.execute_task_space_trajectory(self.get_current_eff_pose(), modified_pos_eef, duration=1)


        return tilt_angle

: 

### Initialize environment and rest

In [ ]:
max_timesteps = 500  # TODO

env = BulletKukaEnvironment()
terminated, truncated = False, False
# observation, info = env.reset()
# env.render()

tilt_angle = .55 # tehta: .001

## generalized position of end-effector: position + orientation (Euler)
third_pos = np.array([[tilt_angle, 0., 0.08], [-np.pi/2, 0., -np.pi/2]])  # right on top of the object, ready to grip the object
fourth_pos = np.array([[tilt_angle, 0., 0.2], [-np.pi/2, 0., -np.pi/2]])  # right on top of the object

init_kuka_joint_angle = np.array([0.]*7)
des_kuka_joint_angle = np.array([0., 0.5734, 0., -2.0801, 0., 0.4265, 0.])  # on top of the object with distance in z-axis

## initial configuration:
env.set_kuka_joint_angles(init_kuka_joint_angle, des_kuka_joint_angle, duration=2)

# z_step = .13
# modified_pos_eef = [list(row) for row in env.get_current_eff_pose()]
# modified_pos_eef[0][2] = env.get_current_eff_pose()[0][2] - z_step
# env.execute_task_space_trajectory(env.get_current_eff_pose(), modified_pos_eef, duration=1)

print(env.get_current_eff_pose())

modified_pos_eef = [list(row) for row in env.get_current_eff_pose()]
x_step = .02
z_step = .13

# modified_pos_eef[0][0] = env.get_current_eff_pose()[0][0] + x_step
# env.execute_task_space_trajectory(env.get_current_eff_pose(), modified_pos_eef, duration=1)
env.execute_gripper(init_pos=0.0, fin_pos=0.01, duration=1)  # open gripper
modified_pos_eef[0][2] = env.get_current_eff_pose()[0][2] - z_step
env.execute_task_space_trajectory(env.get_current_eff_pose(), modified_pos_eef, duration=1)
env.execute_gripper(init_pos=0.0, fin_pos=0.01, duration=1)  # open gripper
modified_pos_eef[0][2] = env.get_current_eff_pose()[0][2] + z_step
env.execute_task_space_trajectory(env.get_current_eff_pose(), modified_pos_eef, duration=1)

print(env.get_current_eff_pose())

# env.step("xPlus")

while True:
    p.stepSimulation()
    time.sleep(sim_time_step)
    pass

((0.44831149390515107, 6.071763009667455e-06, 0.2110464577885206), (-1.4521315034548228, 0.00010200715766449883, -1.5715752593074288))
((0.44214598482222983, 7.525756273437246e-06, 0.1198822821744253), (-1.4543330496205324, 0.00010155728921123283, -1.5715665066608047))


### Execute a single action

In [ ]:
# action = 0
# observation, reward, done, truncated, info = env.step(action)
# env.render()